In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile("/content/javascript_jest_df.zip", 'r') as f:
  f.extractall(path="/content/javascipt_jest_df_unzipped")

In [ ]:
import pandas as pd

In [ ]:
javascript_folder = ["/content/javascipt_jest_df_unzipped/javascript_jest_df/js_part"]
jest_folder = ["/content/javascipt_jest_df_unzipped/javascript_jest_df/jest_part"]

df = pd.DataFrame({'JavaScript Content': javascript_folder})

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os

In [ ]:
def load_javascript_files(directory):
    javascript_files = []
    for filename in os.listdir(directory):
        if filename.endswith('.js'):  # Check if the file is a JavaScript file
            file_path = os.path.join(directory, filename)
            print("Loading JavaScript file:", filename)
            with open(file_path, 'r') as file:
                javascript_files.append(file.read())
    return javascript_files

def load_jest_files(directory):
    jest_files = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        print("Loading Jest file:", filename)
        with open(file_path, 'r') as jest_file:
            jest_files.append(jest_file.read())
    return jest_files

javascript_files = load_javascript_files(javascript_destiny)

jest_files = load_jest_files(jest_destiny)

df = pd.DataFrame({'tokenized_js': javascript_files, 'tokenized_jest': jest_files})
print("Number of JavaScript files loaded:", len(javascript_files))
print("Number of Jest files loaded:", len(jest_files))

print("\nContent of the first JavaScript file:")
print(javascript_files[0])

print("\nContent of the first Jest file:")
print(jest_files[0])


Loading JavaScript file: js_th.js
Loading JavaScript file: js_sev.js
Loading JavaScript file: js_fiv.js
Loading JavaScript file: js_tent.js
Loading JavaScript file: js_sixt.js
Loading JavaScript file: js_sec.js
Loading JavaScript file: js_ni.js
Loading JavaScript file: js_elev.js
Loading JavaScript file: js_cal.js
Loading JavaScript file: js_eight.js
Loading JavaScript file: js_rd.js
Loading Jest file: jest_6.js
Loading Jest file: jest_2.js
Loading Jest file: jest_1O.js
Loading Jest file: jest_st.js
Loading Jest file: jest_4.js
Loading Jest file: jest_5.js
Loading Jest file: jest_11.js
Loading Jest file: jest_9.js
Loading Jest file: jest_3.js
Loading Jest file: jest_7.js
Loading Jest file: jest_8.js
Number of JavaScript files loaded: 11
Number of Jest files loaded: 11

Content of the first JavaScript file:
import { useState, useEffect } from 'react'
import { TodoProvider } from './contexts'
import './App.css'
import TodoForm from './components/TodoForm'
import TodoItem from './componen

In [ ]:
df.shape

(11, 2)

In [ ]:
import re

def custom_tokenizer(code):
    patterns = [
        r'\b\w+\b',
        r'"(?:\\.|[^"\\])*"|\'(?:\\.|[^\'\\])*\'',
        r'//.*|/\*(?:.|\n)*?\*/',
        r'\b(it|describe|test)\s*\((?:[^)(]|\((?:[^)(]+|\([^)(]*\))*\))*\)',
        r'(it|describe|test)\s*\.\s*(?:each|skip|only|todo)',
        r'\b(expect|toMatch|toEqual|toContain|toBe)\s*\(',
        r'[\[\]{}(),;.]|\+\+|--|&&|\|\||<<|>>|===|!==|=>|\?',
        r'\b\d+(?:\.\d*)?(?:e[+-]?\d+)?\b'
    ]

    patterns = [re.compile(pattern) for pattern in patterns]

    tokens = []
    for pattern in patterns:
        tokens.extend(pattern.findall(code))

    return tokens

javascript_code = "function greet(name) { console.log('Hello, ' + name + '!'); } greet('World');"
jest_code = "it('should greet the world', () => { expect(greet('World')).toBe('Hello, World!'); });"

print("JavaScript tokens:", custom_tokenizer(javascript_code))
print("Jest tokens:", custom_tokenizer(jest_code))

df['js_tokens'] = df['tokenized_js'].apply(lambda x: custom_tokenizer(x))
df['jest_tokens'] = df['tokenized_jest'].apply(lambda x: custom_tokenizer(x))

In [ ]:
df.shape

(11, 2)

In [ ]:
df['tokenized_js']

0     import { useState, useEffect } from 'react'\ni...
1                                                      
2     import { useEffect, useState } from 'react'\ni...
3     import React, { Component } from "react";\n\ne...
4     import React, { useContext, useState } from 'r...
5     import React from 'react'\nimport { useSelecto...
6     import { useCallback, useEffect, useRef, useSt...
7     import { useEffect, useState } from "react";\n...
8     import React from 'react'\nimport ReactDOM fro...
9     import './App.css'\nimport { InputBox } from '...
10    import React from 'react'\nimport ReactDOM fro...
Name: tokenized_js, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

train_tokens = train_df['js_tokens'] + train_df['jest_tokens']
val_tokens = val_df['js_tokens'] + val_df['jest_tokens']

train_tokens_flat = [token for sublist in train_tokens for token in sublist]
val_tokens_flat = [token for sublist in val_tokens for token in sublist]

train_tokens_reshaped = [[token] for token in train_tokens_flat]
val_tokens_reshaped = [[token] for token in val_tokens_flat]

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

encoder.fit(train_tokens_reshaped)

train_encoded = encoder.transform(train_tokens_reshaped)
val_encoded = encoder.transform(val_tokens_reshaped)


KeyError: 'tokenized_js'

In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [ ]:
train_df.shape, val_df.shape

In [ ]:
max_jest_length = 600
start_token_index = 0
end_token_index = 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

input_seq_length = 573
output_seq_length = 573
input_vocab_size = 10000
output_vocab_size = 5000
latent_dim = 256
batch_size = 64
epochs = 50

encoder_inputs = Input(shape=(input_seq_length,))
encoder_embedding = Embedding(input_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(output_seq_length,))
decoder_embedding = Embedding(output_vocab_size, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()


In [ ]:
history = model.fit(
    [train_encoded[:, :-1], train_encoded[:, 1:]],
    tf.one_hot(train_encoded[:, 1:], output_vocab_size),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(
        [val_encoded[:, :-1], val_encoded[:, 1:]],
        tf.one_hot(val_encoded[:, 1:], output_vocab_size)
    )
)


val_loss = model.evaluate(
    [val_encoded[:, :-1], val_encoded[:, 1:]],
    tf.one_hot(val_encoded[:, 1:], output_vocab_size)
)

print("Validation Loss:", val_loss)


In [ ]:
import numpy as np

def generate_jest_file(input_sequence, model, max_jest_length, start_token_index, end_token_index, tokenizer):

    preprocessed_code = preprocess_javascript_code(input_sequence)

    tokenized_input_sequence = tokenizer.texts_to_sequences([preprocessed_code])[0]

    padded_input_sequence = pad_sequences([tokenized_input_sequence], maxlen=max_sequence_length, padding='post')

    encoder_states = model.predict(padded_input_sequence)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = start_token_index
    jest_code = []
    for _ in range(max_jest_length):
        output_tokens, h, c = model.predict([target_seq] + encoder_states)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        jest_code.append(sampled_token_index)

        if sampled_token_index == end_token_index:
            break
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        encoder_states = [h, c]

    return jest_code

input_sequence = "function greet(name) { console.log('Hello, ' + name + '!'); } greet('World');"
generated_jest_code = generate_jest_file(input_sequence, model, max_jest_length, start_token_index, end_token_index, tokenizer)
